# Python toolkit for fluorescence data processing

Author: Yongmin Hu (yongmin.hu@eawag.ch)

Last update: 2022-02-13

Before running the toolkit, please go through checklist_fluorescence_python_toolkit.doc.

If you change the input in any of the code blocks, please re-run all blocks after.

# 0. Import packages

In [5]:
import matplotlib
import os
import numpy as np
import matplotlib.pyplot as plt
import re

from datetime import datetime, timedelta, date
from EEMprocessing import *
from generate_widgets import *
from ipywidgets import interact, interactive, interact_manual
from ipywidgets import Layout, Button, Dropdown, Label, FloatText, Text, IntSlider, Checkbox, Box
from IPython.display import display, clear_output
from traitlets import traitlets


matplotlib.rcParams['figure.dpi'] = 80


-------------------------------------------
# 1. Specify data directory and filename format

In [34]:
# read all filenames in the target directory
filedir_default = '20220407_detectionlimit_test' #Just the default, could be changed

Part1_widgets = Widgets1(filedir_default)
data_selection = Part1_widgets.generate_widgets()
display(data_selection)


Box(children=(VBox(children=(Label(value='Pleae specify the directory of fluorescence data in the text box bel…

-----------------------------------------
# 2. Data preview and parameter selection

In [35]:
# Visualization with interaction
datdir = Part1_widgets.dir_selection.value
filelist = os.listdir(datdir)
datlist = [file for file in filelist if 'PEM.dat' in file]
data_index = []
if Part1_widgets.ts_read_from_filename.value:
    tslist = [get_TS_from_filename(file, ts_format=Part1_widgets.ts_format.value, 
                                   ts_start_position=Part1_widgets.ts_start_position.value-1,
                                   ts_end_position=Part1_widgets.ts_end_position.value) for file in datlist]
    tslist_sortidx = np.argsort(tslist)
    tslist_sorted = [tslist[i] for i in tslist_sortidx]
    datlist = [datlist[i] for i in tslist_sortidx]
else:
    data_index = [file[:-4] for file in datlist]
    
# --------------Create widgets for plotting parameters-------------

intensity_range = (0,3000,100) # (min, max, step）
em_range = (250,600,10)
ex_range = (250,500,10)

ts_format = Part1_widgets.ts_format.value
ts_start_position = Part1_widgets.ts_start_position.value
ts_end_position = Part1_widgets.ts_end_position.value

Part2and3_widgets = Widgets2and3(intensity_range, em_range, ex_range, datdir, datlist, ts_format, ts_start_position,
                 ts_end_position)
tab2, note_step2, out_parameters = Part2and3_widgets.generate_widgets()

display(tab2, note_step2, out_parameters)


Output()

----------------------------------------------------
# 3. Stack data

For further analysis, the parameters would be set according to the settings in the above interface!

PLEASE CHECK THE SETTINGS before going-on!


In [15]:
#-------------------------Stack data------------------------------
stacking_interact = Part2and3_widgets.generate_widgets2()
caption = ipywidgets.Label(value='The stack includes all fluorescence data whose filename contains the specified keyword.')
display(caption, stacking_interact)

Label(value='The stack includes all fluorescence data whose filename contains the specified keyword.')

interactive(children=(Text(value='PEM.dat', description='Filename searching keyword: ', style=DescriptionStyle…

------------------------------------------
# 4. Remove unwanted data from the data stack

Please run the code block even if you don't want to do any operation here.

In [16]:
#------------------Remove specific data from the data stack-----------------

eem_stack_all, em_range_cw, ex_range_cw, datlist_all = stacking_interact.result

Part4_widgets = Widgets4(eem_stack_all, datlist_all, em_range_cw, ex_range_cw, Part2and3_widgets.preview_parameter_dict)

manual_cleaning = Part4_widgets.generate_widgets_1()

# auto cleaning
auto_cleaning = Part4_widgets.generate_widgets_2()


tab4 = ipywidgets.Tab()
tab4.children = [manual_cleaning, auto_cleaning]
tab4.set_title(0, 'Manual cleaning')
tab4.set_title(1, 'Auto cleaning')


caption = ipywidgets.Label(
    'This block is used to remove specific data from the data stack. Please select the data you want to remove:')
caption2 = ipywidgets.Label(
    "To reset the data stack to its original state: re-run the block. If you don't want to make any changes here,"
    " please skip.")

display(caption, tab4, caption2)


Label(value='This block is used to remove specific data from the data stack. Please select the data you want t…

Label(value="To reset the data stack to its original state: re-run the block. If you don't want to make any ch…

"2022-01-20-12-03-33-B1C1S1M3R0PEM.dat" has been removed


-------------------------------------
# 5. Data stack analysis

In [17]:
%matplotlib inline
#--------------------Data stack analysis---------------
pd.set_option('display.max_rows', None)
datlist_filtered = Part4_widgets.datlist_filtered
idx2remove = Part4_widgets.idx2remove

# Are there any changes in the last step?
data_index_cw = []

if datlist_filtered==datlist_all:
    datlist_cw = datlist_all[:]
    eem_stack_cw = eem_stack_all[:,:,:]
else:
    datlist_cw = datlist_filtered[:]
    eem_stack_cw = np.delete(eem_stack_all, idx2remove, 0)
    
if Part1_widgets.ts_read_from_filename.value:
    timestamps_cw = [get_TS_from_filename(dat, Part1_widgets.ts_format.value, 
                                          Part1_widgets.ts_start_position.value-1, 
                                          Part1_widgets.ts_end_position.value)
                for dat in datlist_cw ]
    timestamps_cw_sortidx = np.argsort(timestamps_cw)
    timestamps_cw = [timestamps_cw[i] for i in timestamps_cw_sortidx]
    datlist_cw = [datlist_cw[i] for i in timestamps_cw_sortidx]
    eem_stack_cw = eem_stack_cw[timestamps_cw_sortidx]
else:
    data_index_cw = [file[:-4] for file in datlist_cw]
    timestamps_cw = False

#---------------Create widgets------------------------
#----------------General widgets------------
out = ipywidgets.Output()
display(out)

button_clear_display=ipywidgets.Button(description='Clear display')
def clear_display_with_button(b):
    clear_output()
    display(tab)
button_clear_display.on_click(clear_display_with_button)

#-------Tab1: File range selection----------

Part51_widgets = Widgets51(datlist_cw)
data_range_items, range1, range2 = Part51_widgets.generate_widgets()

data_range = Box(data_range_items,
                 layout=Layout(display='flex', flex_flow='column', border='none', align_items='stretch', width='80%'))


#-------Tab2: Pixel statistics------------------

Part52_widgets = Widgets52(eem_stack_cw, datlist_cw, range1, range2, em_range_cw, ex_range_cw, timestamps_cw)
pixel_statistics_items = Part52_widgets.generate_widgets()

pixel_statistics_items.append(ipywidgets.Box([button_clear_display]))
pixel_statistics = Box(pixel_statistics_items, 
                       layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='100%'))

#-------Tab3: EEM statistics---------

Part53_widgets = Widgets53(eem_stack_cw, datlist_cw, range1, range2, em_range_cw, ex_range_cw, timestamps_cw, 
                           Part2and3_widgets.crange_cw)
eem_statistics_items = Part53_widgets.generate_widgets()

eem_statistics_items.append(ipywidgets.Box([button_clear_display]))

eem_statistics = Box(eem_statistics_items, 
                     layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='100%'))

#-------Tab4: Region integration------------

# region_integration_items = [
#     ipywidgets.Box([Label(value='Under construction')]),
#     ipywidgets.Box([button_clear_display], layout=form_item_layout),
# ]

# region_integration = Box(region_integration_items, 
#                          layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='60%'))

Part54_widgets = Widgets54(eem_stack_cw, datlist_cw, range1, range2, em_range_cw, ex_range_cw, timestamps_cw)

integration_items = Part54_widgets.generate_widgets()
integration_items.append(ipywidgets.Box([button_clear_display]))
region_integration = Box(integration_items, 
                         layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='60%'))


#-------Tab5: Stack decomposition----------

Part55_widgets = Widgets55(data_index, data_index_cw, timestamps_cw, eem_stack_cw, datlist_cw, range1, 
                         range2, em_range_cw, ex_range_cw)

decomposition_items = Part55_widgets.generate_widgets()

decomposition = Box(decomposition_items, 
                    layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='100%'))


# -------Tab6: Data reconstruction----------

Part56_widgets = Widgets56(Part55_widgets.decomposition_method_list, Part55_widgets.rank_display, Part2and3_widgets.crange_cw, 
                           data_index, data_index_cw, timestamps_cw, eem_stack_cw, datlist_cw, range1, 
                           range2, em_range_cw, ex_range_cw)
decomposition_reconstruction_items = Part56_widgets.generate_widgets()
decomposition_reconstruction_items.append(ipywidgets.Box([button_clear_display]))

decomposition_reconstruction = Box(decomposition_reconstruction_items, 
                                   layout=Layout(display='flex',flex_flow='column',border='none',align_items='stretch',width='80%'))




# -----------------Assembly tabs-------------------

tab = ipywidgets.Tab() 
tab.children=[data_range, pixel_statistics, eem_statistics, region_integration,
              decomposition, decomposition_reconstruction]
# tab2 = ipywidgets.Tab()
# tab2.children = [decomposition_reconstruction]
# tab.children=[data_range, pixel_statistics, eem_statistics, region_integration,
#               decomposition, tab2]
tab.set_title(0, '1. Data range selection')
tab.set_title(1, '2. Pixel statistics')
tab.set_title(2, '3. EEM statistics')
tab.set_title(3, '4. Region integraion')
tab.set_title(4, '5. Stack decomposition')
tab.set_title(5, '6. Stack reconstruction')
display(tab)

Output()

# 6. Save PARAFAC result

In [19]:
score_df, ex_df, em_df = Part55_widgets.button_decomposition_interact.result
Part6_widgets = Widgets6(score_df, ex_df, em_df, Part1_widgets.dir_selection.value, Part2and3_widgets.inner_filter_effect, 
                         Part2and3_widgets.scattering_correction, Part2and3_widgets.gaussian_smoothing, 
                         Part55_widgets.decomposition_method_list, Part55_widgets.dataset_normalization)
output_items = Part6_widgets.generate_widgets()
decomposition_output = Box(output_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='none',
    align_items='stretch',
    width='80%'
))
display(decomposition_output)

Box(children=(Box(children=(Label(value='Mandatory fields are marked with *'),)), Box(children=(Text(value='sa…

# Test

In [11]:
score_df

component 1  component 2  \
type  time                                                          
Score 2022-01-19-00-18-05-B1C2S1M3R0PEM    46.863317    10.458684   
      2022-01-19-00-45-27-B1C3S1M3R0PEM    46.822728    10.519633   
      2022-01-19-01-12-49-B1C4S1M3R0PEM    46.750975    10.277053   
      2022-01-19-01-48-37-B1C1S1M3R0PEM    47.635987    10.652757   
      2022-01-19-02-15-49-B1C2S1M3R0PEM    46.530929    10.591270   
      2022-01-19-02-43-12-B1C3S1M3R0PEM    46.511383    10.526218   
      2022-01-19-03-10-34-B1C4S1M3R0PEM    46.595145    10.775671   
      2022-01-19-03-46-22-B1C1S1M3R0PEM    46.805338    10.325375   
      2022-01-19-04-13-34-B1C2S1M3R0PEM    46.713546    10.638153   
      2022-01-19-04-40-56-B1C3S1M3R0PEM    46.679051    10.764068   
      2022-01-19-05-08-18-B1C4S1M3R0PEM    46.572258    10.328575   
      2022-01-19-05-44-05-B1C1S1M3R0PEM    46.757683    10.546058   
      2022-01-19-06-11-18-B1C2S1M3R0PEM    46.341023    10.613973   
      2022-01-19-06-38-40-B1C3S1M3R0PEM    47.052740    10.648933   
      2022-01-19-07-06-02-B1C4S1M3R0PEM    46.610809    10.650681   
      2022-01-19-07-41-49-B1C1S1M3R0PEM    46.840675    10.364131   
      2022-01-19-08-09-02-B1C2S1M3R0PEM    46.809478    10.423253   
      2022-01-19-08-36-24-B1C3S1M3R0PEM    46.108766    10.404895   
      2022-01-19-09-03-46-B1C4S1M3R0PEM    46.507218    10.360289   
      2022-01-19-09-39-33-B1C1S1M3R0PEM    47.537228    10.641934   
      2022-01-20-12-19-33-B1C1S1M3R0PEM    85.227360    45.988573   
      2022-01-20-12-33-22-B1C2S1M3R0PEM    87.087905    37.086065   
      2022-01-20-12-47-09-B1C3S1M3R0PEM    83.590157    33.962527   
      2022-01-20-13-00-56-B1C4S1M3R0PEM    85.065502    34.931670   
      2022-01-20-13-22-41-B1C1S1M3R0PEM    85.672073    45.583716   
      2022-01-20-13-36-28-B1C2S1M3R0PEM    89.089781    49.261806   
      2022-01-20-13-50-15-B1C3S1M3R0PEM    91.793134    52.132896   
      2022-01-20-14-04-03-B1C4S1M3R0PEM    94.610956    55.958045   
      2022-01-20-14-25-48-B1C1S1M3R0PEM    94.883027    56.159360   
      2022-01-20-14-39-35-B1C2S1M3R0PEM    94.173582    55.777371   

                                         component 3  component 4  
type  time                                                         
Score 2022-01-19-00-18-05-B1C2S1M3R0PEM     5.536655     6.518833  
      2022-01-19-00-45-27-B1C3S1M3R0PEM     5.578414     6.631211  
      2022-01-19-01-12-49-B1C4S1M3R0PEM     4.897145     6.543495  
      2022-01-19-01-48-37-B1C1S1M3R0PEM     5.217635     6.461212  
      2022-01-19-02-15-49-B1C2S1M3R0PEM     5.938342     6.556800  
      2022-01-19-02-43-12-B1C3S1M3R0PEM     5.811032     6.388993  
      2022-01-19-03-10-34-B1C4S1M3R0PEM     5.777328     6.599650  
      2022-01-19-03-46-22-B1C1S1M3R0PEM     4.502192     6.865573  
      2022-01-19-04-13-34-B1C2S1M3R0PEM     5.024679     6.691919  
      2022-01-19-04-40-56-B1C3S1M3R0PEM     5.453874     6.657304  
      2022-01-19-05-08-18-B1C4S1M3R0PEM     4.713322     6.878207  
      2022-01-19-05-44-05-B1C1S1M3R0PEM     4.614601     6.609114  
      2022-01-19-06-11-18-B1C2S1M3R0PEM     5.350108     6.677576  
      2022-01-19-06-38-40-B1C3S1M3R0PEM     5.531920     6.760737  
      2022-01-19-07-06-02-B1C4S1M3R0PEM     5.168937     6.518746  
      2022-01-19-07-41-49-B1C1S1M3R0PEM     4.895802     6.860049  
      2022-01-19-08-09-02-B1C2S1M3R0PEM     5.231962     6.702188  
      2022-01-19-08-36-24-B1C3S1M3R0PEM     5.183950     6.928269  
      2022-01-19-09-03-46-B1C4S1M3R0PEM     4.905838     6.578248  
      2022-01-19-09-39-33-B1C1S1M3R0PEM     4.487237     6.508983  
      2022-01-20-12-19-33-B1C1S1M3R0PEM    48.431465    13.148967  
      2022-01-20-12-33-22-B1C2S1M3R0PEM    37.361648    13.011599  
      2022-01-20-12-47-09-B1C3S1M3R0PEM    36.537484    11.936867  
      2022-01-20-13-00-56-B1C4S1M3R0PEM    42.431113    10.875227  
      2022-01-20-13-22-41-B1C1S1M3R0PEM    54.939315    10.92704

In [12]:
ex_df

component 1   component 2   component 3  component 4
type wavelength                                                       
Ex   250         1.148131e+00  2.439389e+00  2.697750e+00     3.650532
     252         1.257522e+00  2.086862e+00  2.794871e+00     3.614241
     254         1.389880e+00  1.733517e+00  2.923403e+00     3.378104
     256         1.544928e+00  1.376625e+00  3.032443e+00     3.133267
     258         1.708584e+00  1.064301e+00  3.092491e+00     2.774642
     260         1.873512e+00  7.761801e-01  3.094229e+00     2.445332
     262         2.040797e+00  5.282131e-01  3.023742e+00     2.084999
     264         2.211785e+00  3.998330e-01  2.849830e+00     1.622833
     266         2.383809e+00  2.712740e-01  2.621873e+00     1.243675
     268         2.544628e+00  1.472481e-01  2.355300e+00     0.862394
     270         2.683780e+00  4.763316e-02  2.105566e+00     0.381467
     272         2.795360e+00  1.029067e-02  1.746850e+00     0.008548
     274         2.860294e+00  3.557403e-03  1.366730e+00     0.000201
     276         2.887061e+00  4.015971e-03  1.020701e+00     0.000011
     278         2.884321e+00  1.091297e-02  6.390936e-01     0.000003
     280         2.858137e+00  6.198708e-02  2.132816e-03     0.000004
     282         2.647448e+00  5.002795e-01  7.167218e-07     0.000009
     284         2.361925e+00  1.092550e+00  6.268054e-11     0.000002
     286         2.094557e+00  1.360141e+00  2.267980e-15     0.000009
     288         1.819436e+00  1.612517e+00  1.606157e-17     0.000099
     290         1.492460e+00  1.926896e+00  9.921924e-21     0.003102
     292         1.153182e+00  2.235878e+00  1.248791e-22     0.053666
     294         8.420803e-01  2.343404e+00  2.109106e-22     0.771956
     296         6.084412e-01  2.455148e+00  2.425892e-20     1.163991
     298         4.169908e-01  2.574885e+00  3.346723e-18     1.452229
     300         2.817297e-01  2.655005e+00  7.676478e-16     1.682183
     302         1.906012e-01  2.704370e+00  4.280082e-14     1.853263
     304         1.362658e-01  2.714595e+00  6.079242e-13     1.991838
     306         9.373094e-02  2.713507e+00  4.519257e-12     2.101822
     308         6.525734e-02  2.690420e+00  1.730082e-11     2.191604
     310         4.587890e-02  2.649152e+00  3.813909e-11     2.265072
     312         3.520694e-02  2.584356e+00  5.081077e-11     2.325807
     314         2.340633e-02  2.511192e+00  7.745765e-11     2.380858
     316         1.496117e-02  2.421278e+00  1.047911e-10     2.431701
     318         8.804883e-03  2.314971e+00  1.407497e-10     2.477401
     320         4.546225e-03  2.196684e+00  1.948184e-10     2.518291
     322         3.094767e-03  2.073980e+00  2.810290e-10     2.556858
     324         1.599090e-03  1.941907e+00  2.998081e-10     2.595354
     326         7.662937e-04  1.800297e+00  2.945179e-10     2.630619
     328         3.558781e-04  1.652655e+00  2.549443e-10     2.668853
     330         2.454860e-04  1.511844e+00  1.390747e-10     2.697098
     332         1.089940e-04  1.363414e+00  8.504136e-11     2.732025
     334         4.785469e-05  1.218313e+00  6.129717e-11     2.759642
     336         2.283234e-05  1.082283e+00  4.236201e-11     2.775013
     338         2.080409e-05  9.673245e-01  2.119192e-11     2.770810
     340         1.323571e-05  8.491268e-01  1.353715e-11     2.770483
     342         8.024224e-06  7.349550e-01  1.291452e-11     2.758955
     344         5.194545e-06  6.282014e-01  7.457500e-12     2.733422
     346         4.789613e-06  5.420678e-01  5.605958e-12     2.686062
     348         3.429948e-06  4.518692e-01  4.965357e-12     2.648196
     350         2.997424e-06  3.698684e-01  8.386629e-12     2.607263
     352         2.833088e-06  2.959348e-01  6.786561e-12     2.559601
     354         3.392893e-06  2.394041e-01  9.005960e-12     2.491713
     356         3.596701e-06  1.787666e-01  7.249321e-12     2.429463
     358         3.866470e-06  1.249624e-0